install libraries

In [ ]:
!pip install tensorflow
!pip install Tokenizer
!pip install keras

## import Libraries

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import matplotlib.pyplot  as plt
from sklearn.preprocessing import LabelEncoder


## Load emails and labels

In [ ]:

from google.colab import drive
drive.mount('/content/drive')
data = pd.read_csv('spam.csv',encoding='latin-1')
data.isna().sum()
data.dropna(axis=1, inplace=True)
data

##  data spliting into Features and labels

In [ ]:
x=data['v2']
y=data['v1']
label_encoder = LabelEncoder()


labels frequancy

In [ ]:
plt.figure(facecolor='silver',figsize=(6,6))
plt.hist(y)

In [ ]:
y = label_encoder.fit_transform(y)
X_train,X_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=400)

In [ ]:
tokenizer = Tokenizer(oov_token='<OOV>') #may there is words not seen in x_train so model will not put to it sequance umber in testing
tokenizer.fit_on_texts(X_train)
X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_test_sequences = tokenizer.texts_to_sequences(X_test)
max_sequence_length = max([len(seq) for seq in X_train_sequences])
X_train_padded=pad_sequences(X_train_sequences,maxlen=max_sequence_length)
X_test_padded=pad_sequences(X_test_sequences,maxlen=max_sequence_length)
#count=0
# for i in range (len(X_train_padded[0])):
#   if(X_train_padded[0][i]==0):
#     count+=1
# count

## Build neural network and Fitting the Model

In [ ]:
model=tf.keras.models.Sequential([
    tf.keras.layers.Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=64, input_length=max_sequence_length),#to decrease the dimentinality
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train_padded, y_train, epochs=10, batch_size=32)

# Step 6: Evaluate the Model
test_loss, test_accuracy = model.evaluate(X_test_padded, y_test)

print output of model

In [ ]:
predicted_labels = np.argmax(model.predict(X_test_padded), axis=1)

# Map numeric labels to class names
class_names = ["ham", "spam"]
label_output = []

# Print predicted labels as "ham" or "spam" and concatenate with X_test
for label in predicted_labels:
    label_output.append(class_names[label])
    #print(class_names[label])

new_data = pd.DataFrame({'Label_from_Model': label_output, 'email': X_test})
new_data